<a href="https://colab.research.google.com/github/krishnanraman/felux/blob/main/Felux_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
library(data.table)

In [39]:
rfq = fread("RFQ_Products.csv", header=T)
head(rfq, 10)

Product_Id,Product,Weight_Requested,Destination
<int>,<chr>,<int>,<chr>
1,HR P&O .025 x 48,750000,"Cleveland, OH"
2,"CR DS 0.0158"""" x 48""""",1650000,"Cleveland, OH"
3,"HR CSB 0.096"""" x 60""""",850000,"Cleveland, OH"
4,"GA 050 CHT D G40 0.044"""" x 48""""",100000,"Cleveland, OH"
5,"CR HSLA50 0.0615"""" x 44.57""""",250000,"Dallas, TX"
6,"CR HSLA50 0.0615"""" x 60""""",1350000,"Dallas, TX"
7,"GA 080 NCT O G40 0.0187"""" x 48""""",495000,"Dallas, TX"
8,HR P&O .025 x 48,150000,"Dallas, TX"
9,"CR DS 0.0158"""" x 48""""",100000,"Savannah, GA"


In [5]:
suppliers = fread("Suppliers.csv", header=T)
head(suppliers,10)

Id,Name,QualityFactor
<int>,<chr>,<dbl>
1,Supplier 1,0.20746829
2,Supplier 2,0.23560824
3,Supplier 3,0.23810295
4,Supplier 4,0.09197776
5,Supplier 5,0.36013958
6,Supplier 6,0.37966591
7,Supplier 7,0.42157890
8,Supplier 8,0.21566106
9,Supplier 9,0.43478521


In [6]:
quotes = fread("Quotes.csv", header=T)
head(quotes,10)

Quote_Id,Supplier_Id,Product_Id,Weight_Available,Material_Distance,Lead_Time_Min,Lead_Time_Max,MaterialCost
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1,594398,3962,15,38,235539
2,1,2,1650000,3962,15,38,653838
3,1,3,673651,3962,15,38,266944
4,1,4,79253,3962,15,38,31405
5,1,5,198132,3962,15,38,78512
6,1,6,1069917,3962,15,38,423971
7,1,7,392303,3962,11,38,155456
8,1,8,118879,3962,11,38,47107
9,1,9,79253,3962,11,38,31405


```
What's an ask for a typical product_id, and what weights are available from sellers ?
Lets consider product_id = 1.
How much of it do we need, and what's available on the market ?
```

In [7]:
rfq[rfq$Product_Id == 1]$Weight_Requested
quotes[quotes$Product_Id == 1]$Weight_Available

[1] 750000

[1] 594398 750000 750000 750000 479895 750000 433815 588254 750000 460331


```
Suppose we score sellers by the square distance from the request weight vector. 
Seller with the least square distance can supply most of the goods.
Then we work our way up.

What's the request vector ?
What's the supplier vector from the first supplier ? ( we're assuming each supplier supplies every item )
```

In [8]:
rfq$Weight_Requested
quotes[quotes$Supplier_Id == 1]$Weight_Available

[1]  750000 1650000  850000  100000  250000 1350000  495000  150000  100000
[10]   25000

[1]  594398 1650000  673651   79253  198132 1069917  392303  118879   79253
[10]   19813

```
Lets compute the square distance
```

In [9]:
sqdist <- function(a,b) { round(dist(rbind(a,b))) }
sqdist(rfq$Weight_Requested, quotes[quotes$Supplier_Id == 1]$Weight_Available)

       a
b 385812

In [10]:
res<- mapply( function(x) { sqdist( rfq$Weight_Requested, quotes[quotes$Supplier_Id == x ]$Weight_Available) }, 1:10 ) # Find the squared distance for all 10 suppliers
res <- sort(res, index=T)
res

$x
 [1]  218000  238181  376606  385812  533000  618125  806233  890074  937566
[10] 1025097

$ix
 [1]  4  3  2  1  8  6 10  5  9  7

```
We should start with supplier #4 & then #3,#2,#1,#8 etc in the sort order until our order is fulfilled.
```

In [11]:
need <- rfq$Weight_Requested
mysuppliers <- c()
for (idx in res$ix) {
  cat( "\nNEED\n")
  cat(need)
  cat( paste("\nAvailable from Supplier ", idx, "\n"))
  cat(quotes[quotes$Supplier_Id == idx]$Weight_Available)
  need = mapply( function(x) { max(x,0) }, need - quotes[quotes$Supplier_Id == idx]$Weight_Available)
  mysuppliers <- c( mysuppliers, idx)
  if (sum(need) == 0) {
    cat("\n\nOrder fulfilled! Used the following suppliers to fulfil order: ")
    cat( mysuppliers)
    break
  }
}


NEED
750000 1650000 850000 100000 250000 1350000 495000 150000 100000 25000
Available from Supplier  4 
750000 1498236 771818 90802 227005 1225830 449471 136203 90802 25000
NEED
0 151764 78182 9198 22995 124170 45529 13797 9198 0
Available from Supplier  3 
750000 1650000 647612 100000 250000 1350000 377139 114284 76189 19047

Order fulfilled! Used the following suppliers to fulfil order: 4 3


Produce an Order Fulfillment dataframe


In [38]:
need <- rfq$Weight_Requested
wt_needed <- sum(need)
mysuppliers <- matrix(c("Buyer", need, "100%", ""), ncol=13)
for (idx in res$ix) {
  qty = mapply( function(x,y) { min(x,y) }, need, quotes[quotes$Supplier_Id == idx]$Weight_Available)
  need = mapply( function(x) { max(x,0) }, need - quotes[quotes$Supplier_Id == idx]$Weight_Available)
  coverage = round(sum(qty)*100/wt_needed, 2)
  qual = round(suppliers[suppliers$Id == idx]$QualityFactor, 2)
  mysuppliers <- rbind( mysuppliers, c(paste("Supplier", idx), qty, paste(coverage, " %"), qual))
  if (sum(need) == 0) {
    break
  }
}
df <- data.frame(mysuppliers)
colnames(df) <- c("Party", paste("Product ", rfq$Product_Id), "Coverage", "Quality")
df

Party,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,Product 8,Product 9,Product 10,Coverage,Quality
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Buyer,750000,1650000,850000,100000,250000,1350000,495000,150000,100000,25000,100%,
Supplier 4,750000,1498236,771818,90802,227005,1225830,449471,136203,90802,25000,92.05 %,0.09
Supplier 3,0,151764,78182,9198,22995,124170,45529,13797,9198,0,7.95 %,0.24



**Order fulfilment constrained by quality factor.**
```
Say we seek only those suppliers with a quality > 20%
```

In [40]:
need <- rfq$Weight_Requested
wt_needed <- sum(need)
mysuppliers <- matrix(c("Buyer", need, "100%", ""), ncol=13)
threshold <- 0.2
for (idx in res$ix) {
  qual = round(suppliers[suppliers$Id == idx]$QualityFactor, 2)
  if (qual > threshold) {
    qty = mapply( function(x,y) { min(x,y) }, need, quotes[quotes$Supplier_Id == idx]$Weight_Available)
    need = mapply( function(x) { max(x,0) }, need - quotes[quotes$Supplier_Id == idx]$Weight_Available)
    coverage = round(sum(qty)*100/wt_needed, 2)
    mysuppliers <- rbind( mysuppliers, c(paste("Supplier", idx), qty, paste(coverage, " %"), qual))
    if (sum(need) == 0) {
      break
    }
  }
}
df <- data.frame(mysuppliers)
colnames(df) <- c("Party", paste("Product ", rfq$Product_Id), "Coverage", "Quality")
df

Party,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,Product 8,Product 9,Product 10,Coverage,Quality
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Buyer,750000,1650000,850000,100000,250000,1350000,495000,150000,100000,25000,100%,
Supplier 3,750000,1650000,647612,100000,250000,1350000,377139,114284,76189,19047,93.26 %,0.24
Supplier 2,0,0,202388,0,0,0,117861,35716,23811,5953,6.74 %,0.24


Order fulfilment constrained by shipping distance, quality, and least number of suppliers


In [58]:
indices <- 1:10
res<- mapply( function(x) { sqdist( rfq$Weight_Requested, quotes[quotes$Supplier_Id == x ]$Weight_Available) }, indices ) # Find the squared distance for all 10 suppliers
shippingDistances <- mapply( function(x) { quotes[quotes$Supplier_Id == x ]$Material_Distance[1] }, indices ) # Distances
sortedByShippingDistance <- rbind( indices, res, shippingDistances )[, order( shippingDistances, res)]
cat(" \nSorted by shipping distance, coverage capacity \n")
sortedByShippingDistance

need <- rfq$Weight_Requested
wt_needed <- sum(need)
mysuppliers <- matrix(c("Buyer", need, "100%", ""), ncol=13)
threshold <- 0.2
for (idx in sortedByShippingDistance[1,]) {
  qual = round(suppliers[suppliers$Id == idx]$QualityFactor, 2)
  if (qual > threshold) {
    qty = mapply( function(x,y) { min(x,y) }, need, quotes[quotes$Supplier_Id == idx]$Weight_Available)
    need = mapply( function(x) { max(x,0) }, need - quotes[quotes$Supplier_Id == idx]$Weight_Available)
    coverage = round(sum(qty)*100/wt_needed, 2)
    mysuppliers <- rbind( mysuppliers, c(paste("Supplier", idx), qty, paste(coverage, " %"), qual))
    if (sum(need) == 0) {
      break
    }
  }
}
df <- data.frame(mysuppliers)
colnames(df) <- c("Party", paste("Product ", rfq$Product_Id), "Coverage", "Quality")
cat(paste("\n\n\n Order Fulfilment prioritized by shipping distance, then coverage capacity, thresholded by quality"))

df

 
Sorted by shipping distance, coverage capacity 


indices,9,7,10,6,5,3,2,8,1,4
res,937566,1025097,806233,618125,890074,238181,376606,533000,385812,218000
shippingDistances,2826,2892,3068,3101,3199,3809,3821,3921,3962,4540





 Order Fulfilment prioritized by shipping distance, then coverage capacity, thresholded by quality

Party,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,Product 8,Product 9,Product 10,Coverage,Quality
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Buyer,750000,1650000,850000,100000,250000,1350000,495000,150000,100000,25000,100%,
Supplier 9,750000,932604,850000,56521,141303,763039,495000,84782,56521,25000,72.64 %,0.43
Supplier 7,0,717396,0,43479,108697,586961,0,65218,43479,0,27.36 %,0.42


```
Notes: For Order Fulfilment optimized by price:
a. Since each vendor is only partial-fulfilling an order, the material cost must take partial fulfilment into account. 
b. Shipping distances add to prices as well, given price of fuel miles per gallon. 
```